1. data 불러오기 -> 데이터 불러오는 건 함수로 정의할 것
1. 데이터 준비
    1. 데이터 정제
        1. data 결측치 확인
            1. df.info() 로 null 데이터 여부 확인
            1. 결측치 0 / 평균 / 중간값 으로 채우거나 , 행/열을 삭제
        1. data 이상치 확인 후 수정 / 삭제 (선택사항)
    1. 특성 선택 (선택사항)
        1. 작업에 유용하지 않은 컬럼 삭제
    1. 특성 공학
        1. 연속 특성 이산화
            - 예시)
        1. 특성 분해하기(범주형, 날짜/시간)
            - 예시)
        1. 특성 변환 (log(x), sqrt(x), x^2 등)
        1. 특성 조합으로 새로운 특성 만들필요 있는지 확인
    1. 특성 스케일 조정 (표준화/정규화)






### data 불러오기 -> 데이터 불러오는 건 함수로 정의할 것

In [1]:
# data 불러오기 -> 데이터 불러오는 건 함수로 정의할 것
import os
import pandas as pd
HOUSING_PATH = './datasets/housing'
import numpy as np
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)
df = load_housing_data()

In [2]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
# 데이터 준비
# 데이터 정제
# data 결측치 확인
print(df.info())
# 알 수 있는 것 => 타입, 결측치, 데이터 사이즈
# total_bedrooms 에서 결측치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
None


In [4]:
# ocean_proximity 는 object 타입임로 내용 확인
print(df.ocean_proximity.value_counts())
# 범주형 카테고리임 확인

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64


In [5]:
# 참고만!
# 원래라면 이 단계에서 테스트세트를 나누지만 시험에서는 따로 주어질 것 같으니 패스
# 상식 수준으로 대표적인 샘플링 방식 2개 확인
# 무작위방식
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

# 계층적방식 (테스트 데이터도 원본 데이터의 비율을 유지해야하는 경우)
# NER 에서 각 POS 태그 비율을 맞춘다든지, 인구 데이터에서 남녀 비율을 맞춘다든지, 위 예시에서는 중간 소득을 계층으로 나누어 각 계층 비율을 맞춘다든지하는 방법
print(df.describe())
# 중간 소득을 적당한 개수의 카테고리로 구분 min=0,499, max=15.0001 이고, 3사분위까지가 4.7 정도임을 감안해서 나눔 
df['income_cat'] = pd.cut(df['median_income'], bins=[0.,1.5,3.0,4.5,6.,np.inf], labels=[1,2,3,4,5])
from sklearn.model_selection import StratifiedShuffleSplit
#n_splits = 몇개의 train/test 세트로 나눌 것이냐. 2하면 2개의 train, 2개의 test가 나옴
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# split.split(원본대상, 층화 적용할 시리즈)
for train_index, test_index in split.split(df,df['income_cat']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]
# test 세트 구분을 위해 만든 income_cat은 삭제해서 원본 형태로 복구
strat_train_set = strat_train_set.drop('income_cat',axis=1)
strat_test_set = strat_test_set.drop('income_cat',axis='columns')

          longitude      latitude  housing_median_age   total_rooms  \
count  20640.000000  20640.000000        20640.000000  20640.000000   
mean    -119.569704     35.631861           28.639486   2635.763081   
std        2.003532      2.135952           12.585558   2181.615252   
min     -124.350000     32.540000            1.000000      2.000000   
25%     -121.800000     33.930000           18.000000   1447.750000   
50%     -118.490000     34.260000           29.000000   2127.000000   
75%     -118.010000     37.710000           37.000000   3148.000000   
max     -114.310000     41.950000           52.000000  39320.000000   

       total_bedrooms    population    households  median_income  \
count    20433.000000  20640.000000  20640.000000   20640.000000   
mean       537.870553   1425.476744    499.539680       3.870671   
std        421.385070   1132.462122    382.329753       1.899822   
min          1.000000      3.000000      1.000000       0.499900   
25%        296.00000

층화무작위표본 추출방법의 상세 설명 보고싶으면 여기로 -> [층화무작위표본추출방법](https://rfriend.tistory.com/520) 

In [6]:
# 데이터셋 사이즈가 크지 않다면 특성 간의 표준 상관계수를 살펴봄으로써 중요 특성을 파악해볼 수 있다
# corr() => 피어슨의 r (표준 상관계수 계산)
# median_income과의 양의 상관관계를 확인 가능
corr_matrix = df.corr()
print(corr_matrix['median_house_value'].sort_values(ascending=False))

# 필요하다면 데이터를 조합해서 추가 특성을 생성 (예: 가구당 방의 개수)
df['room_per_household'] = df['total_rooms']/df['households']
new_corr_matrix = df.corr()
print(new_corr_matrix['median_house_value'].sort_values(ascending=False))

median_house_value    1.000000
median_income         0.688075
total_rooms           0.134153
housing_median_age    0.105623
households            0.065843
total_bedrooms        0.049686
population           -0.024650
longitude            -0.045967
latitude             -0.144160
Name: median_house_value, dtype: float64
median_house_value    1.000000
median_income         0.688075
room_per_household    0.151948
total_rooms           0.134153
housing_median_age    0.105623
households            0.065843
total_bedrooms        0.049686
population           -0.024650
longitude            -0.045967
latitude             -0.144160
Name: median_house_value, dtype: float64


## 데이터 준비!

In [15]:
# 테스트 셋에서 예측이 필요한 부분은 따로 떼어놓기 (예측 변수와 타깃값에 변형을 적용하지 않기위함)
df = strat_train_set.drop('median_house_value',axis=1)
df_labels = strat_train_set['median_house_value'].copy()

## 결측값처리!

In [16]:
# 결측치 처리 방법 
# 행삭제, 열삭제, 특정값으로 채우기(0,평균,중간값)
# 중간값 대체 시
from sklearn.impute import SimpleImputer
metric_df = df.drop('ocean_proximity',axis=1)
imputer = SimpleImputer(strategy='median')
# 수치형 자료만 적용가능
imputer.fit(metric_df)
print(imputer.statistics_)
X = imputer.transform(metric_df)
null_filled_df = pd.DataFrame(X, columns=metric_df.columns, index=metric_df.index)

[-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]


## 범주형 데이터 변환!

In [17]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# 머신러닝 알고리즘은 가까이 있는 두 값이 비슷하다고 판단하는 경향이 있음
# 따라서 범주형의 순서 정보가 필요하면 OrdinalEncoder
# 반대는 OneHotEncoder
df_cat = df[['ocean_proximity']]

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(df_cat)

hot_encoder = OneHotEncoder()
housing_cat_1hot = hot_encoder.fit_transform(df_cat)

## 특성 스케일링!

In [18]:
# 표준화/정규화 작업
# 머신러닝 알고리즘은 대부분 입력 숫자 특성들의 스케일이 많이 다르면 잘 작동하지 않음
# 대표적인 정규화: MinMaxScaler, 대표적인 표준화: StandardScaler (이상치 영향을 덜 받음. 평균을 뺀후 표준편차로 나누어 결과 분포의 분산이 1이 되도록 만듦)
# 스케일링은 (테스트 세트가 포함된) 전체 데이터가 아니고, 훈련 데이터에 대해서만 fit() 메서드 적용. 그리고 나서 훈련 세트와 테스트세트에 대해 transform()메서드 적용

# 변환 단계를 순서대로 편하게 처리해주는 Pipeline 이용할 수 있음
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 위에서 적용했던 절차(결측치, 스케일링 등))들을 차례로 ('작업이름', '변환기'), ('작업이름','변환기') 형식으로 입력. 
# 인자의 마지막에는 추정기(fit()만 있는 클래스)나 변환기(fit(),transform() 모두 갖고있는 클래스) 모두 올 수 있고, 그 외에는 모두 변환기만 넣을 수 있음(return 값이 있어야 다음 작업에 넘겨줄 수 있기 때문에?)
metric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# 범주형과 수치형을 한번에 처리하기 위해 ColumnTransformer 사용
from sklearn.compose import ColumnTransformer

# 수치형 자료 컬럼들
metric_attribs = list(metric_df)

# 범주형 자료 컬럼들
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', metric_pipeline, metric_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])
housing_prepared = full_pipeline.fit_transform(df)


In [50]:
print(housing_prepared[:5])

[[-1.15604281  0.77194962  0.74333089 -0.49323393 -0.44543821 -0.63621141
  -0.42069842 -0.61493744  1.          0.          0.          0.
   0.        ]
 [-1.17602483  0.6596948  -1.1653172  -0.90896655 -1.0369278  -0.99833135
  -1.02222705  1.33645936  1.          0.          0.          0.
   0.        ]
 [ 1.18684903 -1.34218285  0.18664186 -0.31365989 -0.15334458 -0.43363936
  -0.0933178  -0.5320456   0.          0.          0.          0.
   1.        ]
 [-0.01706767  0.31357576 -0.29052016 -0.36276217 -0.39675594  0.03604096
  -0.38343559 -1.04556555  0.          1.          0.          0.
   0.        ]
 [ 0.49247384 -0.65929936 -0.92673619  1.85619316  2.41221109  2.72415407
   2.57097492 -0.44143679  1.          0.          0.          0.
   0.        ]]


In [48]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
17606,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,<1H OCEAN
18632,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,<1H OCEAN
14650,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,NEAR OCEAN
3230,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,INLAND
3555,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,<1H OCEAN


In [53]:
strat_test_set.ocean_proximity.value_counts()

<1H OCEAN     1860
INLAND        1288
NEAR OCEAN     534
NEAR BAY       443
ISLAND           3
Name: ocean_proximity, dtype: int64

In [47]:
df.ocean_proximity.value_counts()

<1H OCEAN     7276
INLAND        5263
NEAR OCEAN    2124
NEAR BAY      1847
ISLAND           2
Name: ocean_proximity, dtype: int64

# 이쯤에서 다시 한번 정리하자!
1. 데이터 불러오는 함수 정의해서 dataframe형태로 만들고 
    ```python
    def load_data(): 
        return pd.read_csv('파일경로')
    ```
1. 데이터를 훑어보고, 필요한 데이터 정제가 무엇이 있는지 확인
    1. 데이터 확인 시 간단하게 사용할 수 있는건
        - 전체 자료 구조 간단히 보기: df.info() -> 결측치 확인, 데이터 타입 확인
        - 수치형 자료의 기본 통계값: df.describe() -> 스케일링 필요 여부 확인
        - 상관관계 확인: df.'예측 대상 컬럼명'.corr() -> 새로운 특성 생성 여부 또는 필요/불필요 특성 확인
1. 정제 작업 전에 train_set, test_set 분리.
    - 그냥 단순 분리면 train_test_split
    - 분리 시 계층비율을 유지해야하면 StratifiedShuffleSplit
1. 분리된 train_set 에서 예측이 필요한 컬럼은 따로 떼어 놓기
1. 대표적으로 하는 정제 작업
    1. 결측치 처리 (SimpleImputer(strategy='median'or'mean'or'most_frequent'or'constant'))
        - df.fillna(df.mean()) 이렇게 처리해도 되긴하지만, 나중에 파이프라인으로 있어보이게 처리하려면 sklearn 변환기나 추정기를 활용
    1. 이상치 처리 (이상치는 시각화해서 확인해야할 거 같은데, 시험환경에서는 pyplot 그래프가 안그려지는걸로 보임. 따라서 이상치 자료는 안들어올 수 도...)
    1. 수치형 스케일링 (정규화: MinMax, 표준화: Standard)
    1. 필요 시 새로운 특성 생성 (갖고 있는 자료의 조합 등 처리)
    1. 범주형을 수치형으로 임베딩 (성적A,B,C같은 범주의 순서 가 중요하면 OridnalEncoder, 순서정보를 넣으면 안되면 OneHotEncoder)
1. 위 정제 작업은 사이킷런의 ColumnTransformer 클래스로 보기좋게 처리 가능하다
    ```python
    full_pipleline = ColumnTransformer(
        [
            ('다른작업과 구분가능한 아무이름1', 변환기/추정기/파이프라인 등 fit_transform()함수를 갖고 있는 사이킷런 클래스, 대상 컬럼명 리스트),
            ('다른작업과 구분가능한 아무이름2', 변환기/추정기/파이프라인 등 fit_transform()함수를 갖고 있는 사이킷런 클래스, 대상 컬럼명 리스트),
        ])
    df_prepared = full_pipleline.fit_transform(
        '예측이 필요한 컬럼이 없는 dataframe => 예를 들어 위 예제에서 예측대상인 median_house_value 컬럼이 빠진 dataframe'
    )
    ```




## 이젠 모델 선택하고 훈련 진행하자!



In [19]:
# 선형 회귀 모델
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
# 훈련
lin_reg.fit(housing_prepared, df_labels)

# 예측
housing_predictions = lin_reg.predict(housing_prepared)
# 평가
lin_mse = mean_squared_error(df_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
# 과소 적합 (오차 69050 달러)
lin_rmse

69050.98178244587

In [20]:
# 의사결정트리 모델
from sklearn.tree import DecisionTreeRegressor

# 훈련
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, df_labels)

# 예측
housing_predictions = tree_reg.predict(housing_prepared)
# 평가
tree_rmse = np.sqrt(mean_squared_error(df_labels, housing_predictions))
# 과대 적합 (오차 0)
tree_rmse

0.0

## 과대적합 방지를 위함 교차 검증!

In [21]:
# k-폴드 교차 검증
# 훈련 세트를 폴드(fold)라 불리는 몇개(여기서는 10개!로 해본다)의 서브셋으로 무작위 분할
# 모델을 10번 훈련하고 평가하는데, 매번 다른 폴드를 선택해 평가에 사용하고, 나머지 9개는 훈련에 사용
from sklearn.model_selection import cross_val_score
# 교차 검증 기능은 scoring 매개변수에 (낮을수록 좋은) 비용 함수가 아니라 (클수록 좋은) 효용 함수를 기대함. 그래서 평균제곱오차(MSE)의 반댓값을 계산하는 neg_~ 함수 사용
scores = cross_val_score(tree_reg, housing_prepared, df_labels, scoring='neg_mean_squared_error', cv=10)
# 위에서 음수값이 나왔으므로 제곱근 계산위해 -scores로 부호변경
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

def display_scores(scores):
    print(pd.DataFrame(scores).describe())

In [22]:
display_scores(tree_rmse_scores)

                  0
count     10.000000
mean   69635.974981
std     2937.318761
min    66623.103796
25%    68007.153390
50%    68731.023084
75%    70253.081265
max    76226.285166


In [23]:
# 랜덤포레스트 모델
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
# 훈련
forest_reg.fit(housing_prepared,df_labels)
# 평가
forest_rmse = np.sqrt(mean_squared_error(df_labels, housing_predictions))
print(forest_rmse)
# 교차검증 방식으로 평가
scores = cross_val_score(forest_reg, housing_prepared, df_labels, scoring='neg_mean_squared_error', cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)


0.0
                  0
count     10.000000
mean   49575.218088
std     1923.630000
min    45900.727208
25%    49139.988691
50%    49399.718437
75%    50294.946569
max    53112.355570


## 모델 튜닝하기!
### 하이퍼파리미터 튜닝

In [24]:
# 사이킷런 GridSearchCV 사용하면 가능한 모든 하이퍼파라미터 조합에 대해 교차 검증을 사용해 평가함!
# 사용방법은 아직 잘 모르겠음.. 나중에 다시 정리
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(housing_prepared, df_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [25]:
grid_search.best_params_, grid_search.best_estimator_

({'max_features': 8, 'n_estimators': 30},
 RandomForestRegressor(max_features=8, n_estimators=30))

In [26]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

64967.35283365876 {'max_features': 2, 'n_estimators': 3}
55632.20489023586 {'max_features': 2, 'n_estimators': 10}
53037.59445559791 {'max_features': 2, 'n_estimators': 30}
60651.23795768992 {'max_features': 4, 'n_estimators': 3}
52994.377460975105 {'max_features': 4, 'n_estimators': 10}
50927.38657172976 {'max_features': 4, 'n_estimators': 30}
58958.64709627995 {'max_features': 6, 'n_estimators': 3}
52863.48434515747 {'max_features': 6, 'n_estimators': 10}
50371.07445573919 {'max_features': 6, 'n_estimators': 30}
58125.21269716548 {'max_features': 8, 'n_estimators': 3}
52209.315445054104 {'max_features': 8, 'n_estimators': 10}
50125.01939052624 {'max_features': 8, 'n_estimators': 30}
62717.05709631804 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53858.022281307225 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59816.55736618118 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52827.147141540874 {'bootstrap': False, 'max_features': 3, 'n_estimators'

In [27]:
# 위에서 진행한 랜덤포레스트 모델에서 예측을 하는데 있어 각 특성의 상대적인 중요도를 확인할 수 있음

# 특성 별 중요도 
feature_importances = grid_search.best_estimator_.feature_importances_

# 파이프라인에서 범주형 임베딩 시 사용한 변환기 가져오기
cat_encoder = full_pipeline.named_transformers_['cat']
# 범주형의 원래 이름 확인
cat_attribs_values = cat_encoder.categories_[0]
# 특성의 이름들
attributes = metric_attribs + list(cat_attribs_values)
# 중요도 순으로 나열
sorted(zip(feature_importances, attributes), reverse=True)

# median_income (중위소득)이 가장 중요. 범주형은 섬일 경우 외에는 영향이 거의 없음을 확인 가능
# 이 결과를 바탕으로 특성을 추가하거나, 제거하거나, 이상치를 찾아 제외할 수 있음ttributes = metric_attribs + list(cat_attribs_values)
# 중요도 순으로 나열
sorted(zip(feature_importances, attributes), reverse=True)

# median_income (중위소득)이 가장 중요. 범주형은 섬일 경우 외에는 영향이 거의 없음을 확인 가능
# 이 결과를 바탕으로 특성을 추가하거나, 제거하거나, 이상치를 찾아 제외할 수 있음



[(0.45109413211586025, 'median_income'),
 (0.14056186831602163, 'INLAND'),
 (0.11141641985612545, 'longitude'),
 (0.09890646494738564, 'latitude'),
 (0.050550432260563445, 'housing_median_age'),
 (0.03873361589968171, 'population'),
 (0.029643002746953293, 'total_rooms'),
 (0.029369743384831996, 'total_bedrooms'),
 (0.02612964467668468, 'households'),
 (0.012311110259279017, '<1H OCEAN'),
 (0.0064621047926241145, 'NEAR OCEAN'),
 (0.004784919147006682, 'NEAR BAY'),
 (3.6541596982095907e-05, 'ISLAND')]

## 테스트 세트로 시스템 평가하기!

In [28]:
# 테스트 세트에서 예측 변수와 레이블 추출한 후 full_pipeline으로 데이터 변환하고 (테스트 세트에서 훈련하면 안되니 transform() 호출 => 사실 의도는 알겠지만 이 부분이 잘 이해안감.. 모델학습도 아닌데 변환할 때 fit을 하면안되는 이유는???)
final_model = grid_search.best_estimator_
x_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value'].copy()

x_test_prepared = full_pipeline.transform(x_test)
final_predictions = final_model.predict(x_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse


47841.248463277276

In [33]:
housing_prepared.shape, x_test_prepared.shape

((16512, 13), (4128, 13))

In [34]:
print(housing_prepared)

[[-1.15604281  0.77194962  0.74333089 ...  0.          0.
   0.        ]
 [-1.17602483  0.6596948  -1.1653172  ...  0.          0.
   0.        ]
 [ 1.18684903 -1.34218285  0.18664186 ...  0.          0.
   1.        ]
 ...
 [ 1.58648943 -0.72478134 -1.56295222 ...  0.          0.
   0.        ]
 [ 0.78221312 -0.85106801  0.18664186 ...  0.          0.
   0.        ]
 [-1.43579109  0.99645926  1.85670895 ...  0.          1.
   0.        ]]


In [90]:
# 신뢰구간 계산 방법이라는데.. 전혀 모르겠다아아아ㅏㅇ
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) **2
np.sqrt(stats.t.interval(confidence, len(squared_errors)-1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))

array([45478.10867398, 49296.43640806])